### Importing the Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import time 

### Importing the Dataset

In [2]:
titanic = pd.read_csv('Data/titanic.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
titanic.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


## Data Preprocessing

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [5]:
X = titanic.drop(["Name", "Survived"], axis=1)
y = titanic["Survived"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
titanic.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [9]:
numeric_atr = ["Age", "Fare"]
categorical_atr = ["Pclass", "Sex", "Embarked"]

In [10]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),  # Impute missing values with mean
    ("scaler", StandardScaler())  # Scale numerical features
])

In [11]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Impute missing values with mode
    ("onehot", OneHotEncoder(handle_unknown="ignore"))  # One-hot encode categorical features
])

In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_atr),
        ("cat", categorical_transformer, categorical_atr)
    ])

In [13]:
# Apply preprocessing to training data
X_train_preprocessed = preprocessor.fit_transform(X_train)

# Apply preprocessing to testing data
X_test_preprocessed = preprocessor.transform(X_test)

In [50]:
import joblib
joblib.dump(preprocessor, "objects/processor.joblib")

['objects/processor.joblib']

### Enabling GPU - If you dont have a gpu pls skip to Training 

In [14]:
import tensorflow as tf

In [16]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

### Training the Model and Exporting to the Tensorboard

In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_preprocessed.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [54]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [55]:
epochs = 500
batch_size = 32
activation = "relu"
intial = "relu"
name = f"titanic_train_{epochs}_{batch_size}_{8}_{activation}_{intial}_{int(time.time())}"

In [56]:
# Train the model
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{name}", histogram_freq=1)
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", restore_best_weights=True)
model.fit(X_train_preprocessed, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[tensorboard_callback])

Epoch 1/500
20/20 [==============================] - 2s 20ms/step - loss: 0.7813 - accuracy: 0.6156 - val_loss: 0.5711 - val_accuracy: 0.8472
Epoch 2/500
20/20 [==============================] - 0s 10ms/step - loss: 0.5567 - accuracy: 0.7625 - val_loss: 0.5370 - val_accuracy: 0.8750
Epoch 3/500
20/20 [==============================] - 0s 8ms/step - loss: 0.5568 - accuracy: 0.7516 - val_loss: 0.5349 - val_accuracy: 0.8750
Epoch 4/500
20/20 [==============================] - 0s 8ms/step - loss: 0.5136 - accuracy: 0.7688 - val_loss: 0.5164 - val_accuracy: 0.8750
Epoch 5/500
20/20 [==============================] - 0s 10ms/step - loss: 0.5200 - accuracy: 0.7734 - val_loss: 0.5059 - val_accuracy: 0.8750
Epoch 6/500
20/20 [==============================] - 0s 8ms/step - loss: 0.4790 - accuracy: 0.7953 - val_loss: 0.4983 - val_accuracy: 0.8750
Epoch 7/500
20/20 [==============================] - 0s 10ms/step - loss: 0.4600 - accuracy: 0.7969 - val_loss: 0.4856 - val_accuracy: 0.8889
Epoch 8/5

In [57]:
name

'titanic_train_500_32_8_relu_relu_1692464561'

In [58]:
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.4878 - accuracy: 0.8212


In [59]:
model.save(f"Models/titan_{name}")

INFO:tensorflow:Assets written to: Models/titan_titanic_train_500_32_8_relu_relu_1692464561\assets


INFO:tensorflow:Assets written to: Models/titan_titanic_train_500_32_8_relu_relu_1692464561\assets


In [60]:
loaded_model = tf.keras.models.load_model(f"Models/titan_1")

In [61]:
# Evaluate the loaded model on the test data
test_loss, test_accuracy = loaded_model.evaluate(X_test_preprocessed, y_test)
print(f"Test accuracy (loaded model): {test_accuracy:.4f}")

6/6 [==============================] - 0s 2ms/step - loss: 0.4533 - accuracy: 0.8156
Test accuracy (loaded model): 0.8156


In [52]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
